Disable all I/O so we can see the performance of the libraries not bound by I/O

In [24]:
import sys

class NullIO:
    """A file-like object that discards all writes"""
    def write(self, *args, **kwargs):
        pass
    def flush(self, *args, **kwargs):
        pass
    def close(self, *args, **kwargs):
        pass
    def __enter__(self):
        return self
    def __exit__(self, *args):
        pass

# Replace stdout and stderr with null I/O
sys.stdout = NullIO()
sys.stderr = NullIO()

### Don’t integrate

In [25]:
import structlog
import logging

#### Testing structlog

In [26]:
struct_logger = structlog.get_logger()

In [27]:
%%timeit -r 3 -n 100000
struct_logger.info("Hello World!")  # 3.1s

### Testing stdlib

In [28]:
stdlib_logger = logging.getLogger()

In [29]:
%%timeit -r 3 -n 100000
stdlib_logger.info("Hello World!")

### [Rendering within structlog](https://www.structlog.org/en/stable/standard-library.html#rendering-within-structlog)

```mermaid
flowchart TD
    User[User]
    structlog[structlog]
    stdlib["Standard Library<br>e.g. logging.StreamHandler"]
    Output[Output]

    User -->|structlog.get_logger().info('foo')| structlog
    User -->|logging.getLogger().info('foo')| stdlib
    structlog -->|logging.getLogger().info("{'event': 'foo'}")| stdlib
    stdlib ==>|Output| Output
```

In [30]:
import structlog

structlog.configure(
    processors=[
        # If log level is too low, abort pipeline and throw away log entry.
        structlog.stdlib.filter_by_level,
        # Add the name of the logger to event dict.
        structlog.stdlib.add_logger_name,
        # Add log level to event dict.
        structlog.stdlib.add_log_level,
        # Perform %-style formatting.
        structlog.stdlib.PositionalArgumentsFormatter(),
        # Add a timestamp in ISO 8601 format.
        structlog.processors.TimeStamper(fmt="iso"),
        # If the "stack_info" key in the event dict is true, remove it and
        # render the current stack trace in the "stack" key.
        structlog.processors.StackInfoRenderer(),
        # If the "exc_info" key in the event dict is either true or a
        # sys.exc_info() tuple, remove "exc_info" and render the exception
        # with traceback into the "exception" key.
        structlog.processors.format_exc_info,
        # If some value is in bytes, decode it to a Unicode str.
        structlog.processors.UnicodeDecoder(),
        # Add callsite parameters.
        structlog.processors.CallsiteParameterAdder(
            {
                structlog.processors.CallsiteParameter.FILENAME,
                structlog.processors.CallsiteParameter.FUNC_NAME,
                structlog.processors.CallsiteParameter.LINENO,
            }
        ),
        # Render the final event dict as JSON.
        structlog.processors.JSONRenderer()
    ],
    # `wrapper_class` is the bound logger that you get back from
    # get_logger(). This one imitates the API of `logging.Logger`.
    wrapper_class=structlog.stdlib.BoundLogger,
    # `logger_factory` is used to create wrapped loggers that are used for
    # OUTPUT. This one returns a `logging.Logger`. The final value (a JSON
    # string) from the final processor (`JSONRenderer`) will be passed to
    # the method of the same name as that you've called on the bound logger.
    logger_factory=structlog.stdlib.LoggerFactory(),
    # Effectively freeze configuration after creating the first bound
    # logger.
    cache_logger_on_first_use=True,
)

In [31]:
import logging
import sys

logging.basicConfig(
    format="%(message)s",
    stream=sys.stdout,
    level=logging.INFO,
)

#### Testing structlog

In [32]:
struct_logger = structlog.get_logger()

In [33]:
%%timeit -r 3 -n 100000
struct_logger.info("Hello World!")

KeyboardInterrupt: 

### Testing stdlib

In [11]:
stdlib_logger = logging.getLogger()

In [12]:
%%timeit -r 3 -n 100000
stdlib_logger.info("Hello World!")

### [Rendering using logging-based formatters](https://www.structlog.org/en/stable/standard-library.html#rendering-using-logging-based-formatters)

```mermaid
flowchart TD
    User[User]
    structlog[structlog]
    stdlib["Standard Library<br>e.g. logging.StreamHandler"]
    Output[Output]

    User -->|structlog.get_logger().info('foo', bar=42)| structlog
    User -->|logging.getLogger().info('foo')| stdlib
    structlog -->|logging.getLogger().info('foo', extra={"bar": 42})| stdlib
    stdlib ==>|Output| Output
```


In [13]:
import structlog

structlog.configure(
    processors=[
        structlog.stdlib.filter_by_level,
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.stdlib.PositionalArgumentsFormatter(),
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
        structlog.processors.UnicodeDecoder(),
        # Transform event dict into `logging.Logger` method arguments.
        # "event" becomes "msg" and the rest is passed as a dict in
        # "extra". IMPORTANT: This means that the standard library MUST
        # render "extra" for the context to appear in log entries! See
        # warning below.
        structlog.stdlib.render_to_log_kwargs,
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
    wrapper_class=structlog.stdlib.BoundLogger,
    cache_logger_on_first_use=True,
)

In [14]:
import logging
import sys

from pythonjsonlogger import jsonlogger

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(jsonlogger.JsonFormatter())
root_logger = logging.getLogger()
root_logger.addHandler(handler)

#### Testing structlog

In [15]:
struct_logger = structlog.get_logger()

In [16]:
%%timeit -r 3 -n 100000
struct_logger.info("Hello World!")

### Testing stdlib

In [17]:
stdlib_logger = logging.getLogger()

In [18]:
%%timeit -r 3 -n 100000
stdlib_logger.info("Hello World!")

### [Rendering using structlog-based formatters within logging](https://www.structlog.org/en/stable/standard-library.html#rendering-using-structlog-based-formatters-within-logging)

```mermaid
flowchart TD
    User[User]
    structlog[structlog]
    stdlib[Standard Library]
    structlog2[structlog]
    stdlib2["Standard Library<br>e.g. logging.StreamHandler"]
    Output[Output]

    User -->|structlog.get_logger().info("foo", bar=42)| structlog
    User -->|logging.getLogger().info("foo")| stdlib
    structlog -->|logging.getLogger().info(event_dict, {"extra": {"_logger": logger, "_name": name})| stdlib
    stdlib -->|structlog.stdlib.ProcessorFormatter.format(logging.Record)| structlog2
    structlog2 -->|Returns a string that is passed into logging handlers.<br>This flow is controlled by the logging configuration.| stdlib2
    stdlib2 ==>|Output| Output
```

In [19]:
import logging
import structlog

structlog.configure(
    processors=[
        # Prepare event dict for `ProcessorFormatter`.
        structlog.stdlib.ProcessorFormatter.wrap_for_formatter,
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
)

formatter = structlog.stdlib.ProcessorFormatter(
    processors=[structlog.dev.ConsoleRenderer()],
)

handler = logging.StreamHandler()
# Use OUR `ProcessorFormatter` to format all `logging` entries.
handler.setFormatter(formatter)
root_logger = logging.getLogger()
root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)

#### Testing structlog

In [20]:
struct_logger = structlog.get_logger()

In [21]:
%%timeit -r 3 -n 100000
struct_logger.info("Hello World!")

### Testing stdlib

In [22]:
stdlib_logger = logging.getLogger()

In [23]:
%%timeit -r 3 -n 100000
stdlib_logger.info("Hello World!")